In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
def shoulder_size(data):
    right_shoulder = [data['x'].loc[2], data['y'].loc[2]]
    left_shoulder = [data['x'].loc[5], data['y'].loc[5]]
    if right_shoulder[0] ==0:
        length = distance.euclidean([data['x'].min(), data['y'].max()] ,left_shoulder)
    elif left_shoulder[0] == 0:
        length = distance.euclidean([data['x'].min(), data['y'].max()] ,right_shoulder)
    elif right_shoulder[0]>0 and left_shoulder[0]>0:
        length = distance.euclidean(right_shoulder, left_shoulder)
    else:
        length=0   
    return(length)

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if_hip_missing = [data['x'].min(), data['y'].max()]
    if neck ==0:
        torso=0
        return(torso)
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0 and hip2[0]>0:
        torso = distance.euclidean(hip2, neck)
    elif hip2[0]==0 and hip1[0]>0:
        torso = distance.euclidean(hip1, neck)
    else:
        torso = distance.euclidean(if_hip_missing, neck)
    return(torso)

In [ ]:
def find_min_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,14,15,16,17]])#remove the arms from the equation cause the bounding boxes are too large 
    x = round(np.min(array[np.nonzero(array)]))
    return(x)

def find_max_round(array):
    array = np.array(array.loc[[0,1,2,5,8,11,14,15,16,17]])
    x = round(np.max(array))
    return(x)

In [ ]:
path_folders = Path('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/cluster_folders')
path_videos = Path('/Users/andreibirladeanu/Documents/Data/meal_videos')

In [ ]:
folders = sorted_alpha(path_folders)
videos = sorted_alpha(path_videos)

In [ ]:
len(folders[0])

In [ ]:
for folder, video in zip(folders, videos):
    print(folder)
    cap = cv2.VideoCapture(os.path.join(path_videos, video))
    samples = []
    skeletons = []
    for skeleton in sorted_alpha(os.path.join(path_folders, folder)):
        skeletons.append(skeleton)
        for coord in sorted_alpha(os.path.join(path_folders,folder,skeleton)):
            cap.set(1, int(coord.split(".")[0])-1)
            ret, image = cap.read()
            if ret == False:
                continue
            data = pd.read_csv(os.path.join(path_folders,folder,skeleton,coord))
            shoulder = shoulder_size(data)
            torso = torso_size(data)
            body = [shoulder,torso]
            img = image[find_min_round(data['y']):find_max_round(data['y']),find_min_round(data['x']):find_max_round(data['x'])]
            if img.shape[0]==0 or img.shape[1]==0:
                samples.append({coord:[(np.average(body, weights=[6./10 ,4./10]))
                                            ,0,0,0]})
                continue
            b,g,r = cv2.split(img)
            samples.append({coord:[(np.average(body, weights=[6./10 ,4./10])),np.mean(b),np.mean(g), np.mean(r)]})

    X = np.array([list(x.values())[0] for x in samples])
    labels = KMeans(n_clusters=2, random_state=0).fit_predict(X)
    new_sk1 = Path(os.path.join(path_folders, folder, '_label_0'))
    new_sk2 = Path(os.path.join(path_folders, folder, '_label_1'))
    new_sk1.mkdir(parents=True,exist_ok=True); new_sk2.mkdir(parents=True,exist_ok=True)
    first_skel = set(sorted_alpha(os.path.join(path_folders, folder, skeletons[0])))
    second_skel = set(sorted_alpha(os.path.join(path_folders, folder, skeletons[1])))
    third_skel = set(sorted_alpha(os.path.join(path_folders, folder, skeletons[2])))
    count = 0
    for label, sample in zip(labels, samples):
        file = list(sample.keys())[0]
        count += 1
        if count <= len(first_skel):
            if file in first_skel:
                if label == 0:
                    os.rename(os.path.join(path_folders,folder,skeletons[0], file) ,os.path.join(new_sk1, file))
                else:
                    os.rename(os.path.join(path_folders,folder,skeletons[0], file) ,os.path.join(new_sk2, file))
        elif count <= len(second_skel):
            if file in second_skel:
                if label == 0:
                    os.rename(os.path.join(path_folders,folder,skeletons[1], file) ,os.path.join(new_sk1, file))
                else:
                    os.rename(os.path.join(path_folders,folder,skeletons[1], file) ,os.path.join(new_sk2, file))
        else:
            if file in third_skel:
                if label == 0:
                    os.rename(os.path.join(path_folders,folder,skeletons[2], file) ,os.path.join(new_sk1, file))
                else:
                    os.rename(os.path.join(path_folders,folder,skeletons[2], file) ,os.path.join(new_sk2, file))
            
    shutil.rmtree(os.path.join(path_folders, folder,skeletons[0]))
    shutil.rmtree(os.path.join(path_folders, folder,skeletons[1]))
        
print('success')       

In [ ]:
print('success')

In [ ]:
a= set([2,4,5])
len(a)

In [ ]:
len(samples)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
for x, sample in zip(X, samples):
    print(kmeans.predict([x])[0])

In [ ]:
list(samples[0].keys())[0].parent

In [ ]:
folders[1:]

## version 2

In [ ]:
### cluster by body size

for folder, video in zip(folders ,videos):
    print(folder)
    
    skeletons = []
    for skeleton in sorted_alpha(os.path.join(path_folders, folder))[1:]:
        if len(skeleton)>6:
            continue
        samples = []
        skeletons.append(skeleton)
        for coord in sorted_alpha(os.path.join(path_folders,folder,skeleton)):
            data = pd.read_csv(os.path.join(path_folders,folder,skeleton,coord))
            shoulder = shoulder_size(data)
            torso = torso_size(data)
            body = [shoulder,torso]
            samples.append([shoulder,torso])
        X = np.array(samples)
        labels =  KMeans(n_clusters=2, random_state=0).fit_predict(X)
        new_sk1 = Path(os.path.join(path_folders, folder, skeleton+'_label_0'))
        new_sk2 = Path(os.path.join(path_folders, folder, skeleton+'_label_1'))
        new_sk1.mkdir(parents=True,exist_ok=True); new_sk2.mkdir(parents=True,exist_ok=True)

        for label,coord_ in zip(labels ,sorted_alpha(os.path.join(path_folders,folder,skeleton))):
            if label==0:
                os.rename(os.path.join(path_folders,folder,skeleton,coord_), os.path.join(new_sk1,coord_))
            else:
                os.rename(os.path.join(path_folders,folder,skeleton,coord_), os.path.join(new_sk2,coord_))
        shutil.rmtree(os.path.join(path_folders, folder,skeleton))


print('success')       

In [ ]:
print('success')

In [ ]:
filename = Path('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2/1043_meal/skel1_label_0/73.csv')

In [ ]:
filename.stem

In [ ]:
filename.suffix

In [ ]:
filename.is_file()

In [ ]:
dest = f"{filename.stem}_1"

In [ ]:
folder_1 = sorted_alpha('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2/1043_meal/skel2_label_0')
folder_2 = sorted_alpha('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2/1043_meal/skel1_label_0')
folder_3 = sorted_alpha('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2/1043_meal/skel2_label_1')
folder_4 = sorted_alpha('/Users/andreibirladeanu/Documents/Data/tracking/skeletons/meal_tracked_missing_joints_2/1043_meal/skel1_label_1')

In [ ]:
u, c = np.unique(f, return_counts=True)
dup = u[c==3]

In [ ]:
f = np.concatenate([folder_1,dup folder_2, folder_3, folder_4])

In [ ]:
dup

In [ ]:
len(f)